In [ ]:
import sys
import os

import cv2
from PyQt5.QtCore import QThread, pyqtSignal, Qt
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtWidgets import QApplication, QWidget, QLabel

from keras.preprocessing import image
from keras.models import load_model
import numpy as np


class Thread1(QThread):
    changePixmap = pyqtSignal(QPixmap)

    def __init__(self, parent):
        QThread.__init__(self, parent=parent)

    def run(self):
        cap = cv2.VideoCapture(0)

        i = 0
        while True:
            ret, frame = cap.read()
            rgbImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            convertToQtFormat = QImage(rgbImage.data, rgbImage.shape[1], rgbImage.shape[0], QImage.Format_RGB888)
            convertToQtFormat = QPixmap.fromImage(convertToQtFormat)
            p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
            self.changePixmap.emit(p)

            if i % 10 == 0:
                self.parent().frame = frame
            i += 1


class Thread2(QThread):
    def __init__(self, parent):
        QThread.__init__(self, parent=parent)

    def run(self):
        emotions = ('злость', 'счастье', 'грусть', 'удивление')
        model_path = ""

        for file in os.listdir(sys.path[0]):
            file_path = os.path.join(sys.path[0], file)
            if os.path.isfile(file_path) & (file.endswith('.h5')):
                model_path = file_path
                self.parent().label5.setText(file)
                model = load_model(model_path)

        i = 0
        while True:
            if model_path and (i % 10 == 0):

                resized_image = cv2.resize(self.parent().frame, (96, 96))
                image_array = np.expand_dims(resized_image, axis = 0) / 255

                predictions = model.predict(image_array)
                max_emotion_percent = '{:.2f}'.format(np.amax(predictions))
                max_emotion_index = np.argmax(predictions)
                recognized_emotion = emotions[max_emotion_index].capitalize() + ' (' + max_emotion_percent + ')'
                self.parent().label3.setText(recognized_emotion)

            i += 1


class App(QWidget):
    
    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 Video'
        self.left = 100
        self.top = 100
        self.width = 640
        self.height = 480
        self.initUI()

    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        self.resize(800, 635)

        self.label1 = QLabel(self)
        self.label1.move(80, 60)
        self.label1.resize(640, 480)

        self.label2 = QLabel(self)
        self.label2.move(340, 520)
        self.label2.resize(100, 100)
        self.label2.setText('Эмоция: ')

        self.label3 = QLabel(self)
        self.label3.move(420, 520)
        self.label3.resize(150, 100)

        self.label4 = QLabel(self)
        self.label4.move(340, 550)
        self.label4.resize(100, 100)
        self.label4.setText('Модель: ')

        self.label5 = QLabel(self)
        self.label5.move(420, 550)
        self.label5.resize(150, 100)

        th1 = Thread1(self)
        th1.changePixmap.connect(lambda p: self.label1.setPixmap(p))
        th1.start()

        th2 = Thread2(self)
        th2.start()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    ex.show()
    app.exec_()